# Visualizations for fast-osvos
## Evaluating both vgg16, resnet18 and resnet34
For all three architectures, both offline and online models are shown. The best performing online models represent their corresponding architectures. The offline models are chosen based on the online models.

In [ ]:
from pathlib import Path
from itertools import product

import matplotlib
matplotlib.use('webagg')
import matplotlib.pyplot as plt
import numpy as np
import yaml
from tqdm import tqdm
from IPython.display import display
import imageio
from moviepy.editor import ImageSequenceClip
from moviepy.video.io.bindings import mplfig_to_npimage
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=False)

In [ ]:
variant_resnet18_offline, variant_resnet18_online = 11, 11
variant_resnet34_offline, variant_resnet34_online = 11, 11

base_path_results = '/home/klaus/dev/fast-osvos/src/results/'
base_path_dataset = '/home/klaus/dev/datasets/DAVIS/'
base_paths = [('Original Image', base_path_dataset + 'JPEGImages/480p/'),
              ('Ground Truth', base_path_dataset + 'Annotations/480p/'),
              ('vgg16', base_path_results + 'vgg16/online/'),
              ('resnet18', base_path_results + 'resnet18/{0}/{1}/'.format(variant_resnet18_offline,
                                                                          variant_resnet18_online)), 
              ('resnet34', base_path_results + 'resnet34/{0}/{1}/'.format(variant_resnet34_offline,
                                                                            variant_resnet34_online))]

models = ['vgg16_offline', 'resnet18_offline', 'resnet34_offline',
          'vgg16_online', 'resnet18_online', 'resnet34_online']

base_path_eval = '/home/klaus/dev/davis-2017-fork/python/tools/output/metrics_mine_'
file_paths_eval = {
    'vgg16_offline': base_path_eval + 'vgg16_offline.yml',
    'resnet18_offline': base_path_eval + 'resnet18_{0}_offline.yml'.format(variant_resnet18_offline),
    'resnet34_offline': base_path_eval + 'resnet34_{0}_offline.yml'.format(variant_resnet34_offline),
    'vgg16_online': base_path_eval + 'vgg16_online.yml',
    'resnet18_online': base_path_eval + 'resnet18_{0}_{1}.yml'.format(variant_resnet18_offline,
                                                                      variant_resnet18_online),
    'resnet34_online': base_path_eval + 'resnet34_{0}_{1}.yml'.format(variant_resnet34_offline,
                                                                      variant_resnet34_offline)
}

In [ ]:
data_speeds = [('vgg16', 0.08083438105769455), 
               ('resnet18', 0.010670146435228262),
               ('resnet34', 0.013862044609998438)]

In [ ]:
def get_metrics_for_model(file_path, metrics):
    with open(file_path, 'r') as stream:
        model_eval = yaml.load(stream)
        
    metrics = [model_eval['dataset'][base][specific]
               for base, specific in metrics]
    
    return metrics

def part_better(specific):
    if specific == 'decay':
        better = '↓'
    else:
        better = '↑'
    return better

def get_metrics_all(models):
    metrics_base = ['J', 'F']
    metrics_specific = ['mean', 'recall', 'decay']
    metrics = list(product(metrics_base, metrics_specific))
    
    metrics_names = [base + '_' + specific + part_better(specific)
                     for base, specific in metrics]
    metrics_all = [(m, get_metrics_for_model(file_paths_eval[m], metrics))
                   for m in models]
    return metrics_names, metrics_all
 
metrics_names, data_metrics_all = get_metrics_all(models)

In [ ]:
def get_metric(file_path, metric, parts):
    with open(file_path, 'r') as stream:
        model_eval = yaml.load(stream)
    keys = sorted(model_eval['sequence'].keys(), reverse=True)
    stats = [[model_eval['sequence'][k][metric][p][0] for p in parts]
             for k in keys]
    part_0, part_1 = list(zip(*stats))
    return part_0, part_1, keys

def get_data_J(file_paths_eval):    
    vgg16_J_mean_offline, vgg16_J_decay_offline, keys = get_metric(file_paths_eval['vgg16_offline'], 
                                                                   'J', ['mean', 'decay'])

    resnet18_J_mean_offline, resnet18_J_decay_offline, _ = get_metric(file_paths_eval['resnet18_offline'],
                                                                      'J', ['mean', 'decay'])

    resnet34_J_mean_offline,  resnet34_J_decay_offline, _ = get_metric(file_paths_eval['resnet34_offline'],
                                                                       'J', ['mean', 'decay'])

    vgg16_J_mean_online, vgg16_J_decay_online, _ = get_metric(file_paths_eval['vgg16_online'],
                                                              'J', ['mean', 'decay'])

    resnet18_J_mean_online, resnet18_J_decay_online, _ = get_metric(file_paths_eval['resnet18_online'],
                                                                    'J', ['mean', 'decay'])

    resnet34_J_mean_online, resnet34_J_decay_online, _ = get_metric(file_paths_eval['resnet34_online'], 
                                                                    'J', ['mean', 'decay'])
    
    data_J_mean = [('vgg16_offline', vgg16_J_mean_offline),
                   ('resnet18_offline', resnet18_J_mean_offline),
                   ('resnet34_offline', resnet34_J_mean_offline),
                   ('vgg16_online', vgg16_J_mean_online), 
                   ('resnet18_online', resnet18_J_mean_online), 
                   ('resnet34_online', resnet34_J_mean_online)]
    
    data_J_decay = [('vgg16_offline', vgg16_J_decay_offline),
                    ('resnet18_offline', resnet18_J_decay_offline), 
                    ('resnet34_offline', resnet34_J_decay_offline),
                    ('vgg16_online', vgg16_J_decay_online), 
                    ('resnet18_online', resnet18_J_decay_online), 
                    ('resnet34_online', resnet34_J_decay_online)]
    
    return keys, data_J_mean, data_J_decay

keys, data_J_mean, data_J_decay = get_data_J(file_paths_eval)

In [ ]:
def plot_bars(data, y_axis_label, title):
    names, speeds = zip(*data)
    data = [go.Bar(y=[speed], x=[name], name=name) for name, speed in data]

    layout = go.Layout(title=title,
                       font=dict(family='Roboto'),
                       xaxis=dict(title='Model', ticks='outside'),
                       yaxis=dict(title=y_axis_label),
                       showlegend=True
                      )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

# The speeds of the three models were evaluated in a similar fashion as specified in the link

https://github.com/jcjohnson/cnn-benchmarks

In [ ]:
plot_bars(data_speeds, y_axis_label='Forward speed per image in seconds',
          title='Average forward speed for each model (lower is better)')

In [ ]:
def plot_metrics_all(keys, data):
    data = [go.Bar(y=y, x=keys, name=name, orientation='v') for name, y in data]

    layout = go.Layout(title=('Metrics for each model'),
                       font=dict(family='Roboto'),
                       xaxis=dict(title='Value', ticks='outside'),
                       yaxis=dict(title='Metric'),
                       showlegend=True,
                       bargap=0.33
                      )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

# An overview of the metrics

In [ ]:
plot_metrics_all(metrics_names, data_metrics_all)

In [ ]:
def plot_metrics(keys, metric, lower_higher, data):
    data = [go.Bar(y=keys, x=x, name=name, orientation='h') for name, x in data]
    
    title = '{metric} per object for each model ({lower_higher} is better)'.format(metric=metric,
                                                                                   lower_higher=lower_higher)
    layout = go.Layout(title=title,
                       font=dict(family='Roboto'),
                       xaxis=dict(title=metric,  ticks='outside', side='top'),
                       yaxis=dict(title='Object'),
                       showlegend=True,
                       height=1500,
                       bargap=0.33,
                       # autosize=False, 
                       margin=dict(pad=10, t=150, l=100)
                      )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

# An overview over J_mean and J_decay for each object

In [ ]:
plot_metrics(keys, 'J_mean', 'higher', data_J_mean)
plot_metrics(keys, 'J_decay', 'lower', data_J_decay)

In [ ]:
def convert_to_rgb(image):
    if len(image.shape) == 2:
        width, height = image.shape
        rgb =  np.empty((width, height, 3), dtype=np.uint8)
        rgb[:, :, :] = image[:, :, None]
        return rgb
    else:
        return image

def dir_to_images(path):
    files = Path(path).iterdir()
    files = map(str, files)
    files = sorted(files)
    files = map(imageio.imread, files)
    files = map(convert_to_rgb, files)
    files = list(files)
    return files

def generate_image(frame_sources, n_rows, n_columns, descriptions):
    width = 8
    height = 8
    fig = plt.figure(figsize=(width, height))
    for image_index, image_source in enumerate(frame_sources):
        ax = plt.subplot(n_rows, n_columns, image_index + 1)
        plt.imshow(image_source)
        plt.axis('off')
        ax.set_title(descriptions[image_index], fontsize=24, fontname='Roboto')
    
    image = mplfig_to_npimage(fig)
    plt.close()
    return image

def display_video(category, base_paths):
    sources = [(desc, path + category) for desc, path in base_paths]
    descriptions, sources = list(zip(*sources))
    sources = list(map(dir_to_images, sources))

    n_columns = 2
    n_rows = len(sources) / n_columns + 1
    sources = list(zip(*sources))
    
    frames = [generate_image(frame_sources, n_rows, n_columns, descriptions) 
              for frame_sources in tqdm(sources)]
    clip = ImageSequenceClip(frames, fps=4)
    
    display(clip.ipython_display(loop=True))
    
def display_category(category, base_paths, data_J_mean, data_J_decay):
    index_data = keys.index(category)
    
    title_format = '{0} for each model of the category {1} ({2} is better)'
    
    data_single_metric = [(desc, metrics[index_data]) for desc, metrics in data_J_mean]
    plot_bars(data_single_metric, y_axis_label='J_mean', 
              title=title_format.format('J_mean', category, 'higher'))
    
    data_single_metric = [(desc, metrics[index_data]) for desc, metrics in data_J_decay]
    plot_bars(data_single_metric, y_axis_label='J_decay', 
              title=title_format.format('J_decay', category, 'lower'))
    
    display_video(category, base_paths)

# Some examples where resnet performs better than vgg16

In [ ]:
display_category('libby', base_paths, data_J_mean, data_J_decay)

In [ ]:
display_category('kite-surf', base_paths, data_J_mean, data_J_decay)

In [ ]:
display_category('horsejump-high', base_paths, data_J_mean, data_J_decay)

# Some examples where vgg16 is still better (mostly the same objects as in older versions)

In [ ]:
display_category('car-shadow', base_paths, data_J_mean, data_J_decay)

In [ ]:
display_category('motocross-jump', base_paths, data_J_mean, data_J_decay)

In [ ]:
import yaml

In [ ]:
base_path_eval = '/home/klaus/dev/davis-2017-fork/python/tools/metrics/'

sequences_val = ['blackswan', 'bmx-trees', 'breakdance', 'camel', 'car-roundabout', 'car-shadow', 'cows',
                 'dance-twirl', 'dog', 'drift-chicane', 'drift-straight', 'goat', 'horsejump-high', 'kite-surf', 
                 'libby', 'motocross-jump', 'paragliding-launch', 'parkour', 'scooter-black', 'soapbox']

In [ ]:
def plot_box(data):
    d = []
    for k, v in sorted(data.items()):
        sequences_names, speeds = list(zip(*v.items()))
        trace = go.Box(
            x = [k / 100] * len(speeds),
            y = speeds,
            name = str(k),
            legendgroup = str(prune_per_iter),
            jitter=0.3,
            boxpoints = False,
        )
        d.append(trace)

    layout = go.Layout(title='Forward speed prune {0}'.format(prune_per_iter),
                       font=dict(family='Roboto'),
                       xaxis=dict(title='Percentage pruned',  ticks='outside', range=(0, 1)),
                       yaxis=dict(title='Forward speed in seconds', range=(0.01, 0.025)),
                       showlegend=True,
                      )
    fig = go.Figure(data=d, layout=layout)
    py.iplot(fig)
    
    
def plot_line(data):
    d = []

    x = []
    y = []
    for k, v in sorted(data.items()):
        sequences_names, speeds = list(zip(*v.items()))
        x.append(k / 100)
        y.append(np.asarray(speeds).mean())

    trace = go.Scatter(
        x = x,
        y = y,
        name = str(prune_per_iter),
        legendgroup = str(prune_per_iter),
    )
    d.append(trace)

    layout = go.Layout(title='Forward speed prune',
                       font=dict(family='Roboto'),
                       xaxis=dict(title='Percentage pruned',  ticks='outside', range=(0, 1)),
                       yaxis=dict(title='Forward speed in seconds', range=(0.01, 0.025)),
                       showlegend=True,
                      )
    fig = go.Figure(data=d, layout=layout)
    py.iplot(fig)
    
    
def plot_metric_per_percentage(data):
    x = [p / 100 for p in prune_percentages]
    y = list(zip(*metrics_all))

    d = []

    for data_percentage, metric_name in zip(y, metrics_names):
        trace = go.Scatter(
            x = x,
            y = data_percentage,
            name = metric_name,
            legendgroup = metric_name[0],
        )
        d.append(trace)

    py.iplot(d)
    
    
def get_data_prune():
    with open('prune_64.yml'.format(prune_per_iter)) as f:
        data = yaml.safe_load(f)
    return data

    
    
    
    
def plot_prune(data):
    # plot_box(data_prune)
    # plot_line(data_prune)
    
    prune_percentages = range(10, 100, 10)
    prune_evaluations = ['prune_per_iter={0},percentage={1}.yml'.format(prune_per_iter, p)
                         for p in prune_percentages]
    prune_evaluations = [(base_path_eval + 'prune/' + p) for p in prune_evaluations]
    
    
    metrics_base = ['J', 'F']
    metrics_specific = ['mean', 'recall', 'decay']
    metrics = list(product(metrics_base, metrics_specific))

    metrics_names = [base + '_' + specific + part_better(specific)
                     for base, specific in metrics]
    metrics_all = [get_metrics_for_model(p, metrics) for p in prune_evaluations]
    
    plot_metric_per_percentage(data_prune)
    
    

    
plot_prune(get_data_prune())

In [ ]:
def part_better(specific):
    if specific == 'decay':
        better = '↓'
    else:
        better = '↑'
    return better


metrics_base = ['J', 'F']
metrics_specific = ['mean', 'recall', 'decay']
metrics = list(product(metrics_base, metrics_specific))
metrics = [metrics[0]]

metrics_names = [base + '_' + specific + part_better(specific)
                 for base, specific in metrics]

In [ ]:
def get_data(data, scale, files):
    accuracies = []
    forward_speeds = []
    for a, b in metrics:
        data_metrics = {}
        for pp, f in zip(scale, files):
            with open(f, 'r') as stream:
                model_eval = yaml.load(stream)
            for k, v in model_eval['sequence'].items():
                data_metrics[(pp, k)] = v[a][b][0]

        data_speeds = {}
        for k in data_metrics.keys():
            data_speeds[k] = data[k[0]][k[1]]


        tmp_accs = []
        tmp_fs = []

        for s in scale:
            a = []
            fs = []
            for seq in sequences_val:
                a.append(data_metrics[(s, seq)])
                fs.append(data_speeds[(s, seq)])
            tmp_accs.append(a)
            tmp_fs.append(fs)
            
        accuracies.append(tmp_accs)
        forward_speeds.append(tmp_fs)

    return forward_speeds, accuracies
        

names, scales, fs, accs = [], [], [], []
    
names.append('vgg16')
s = [1]
files = ['vgg16.yml']
files = [(base_path_eval + 'base/' + p) for p in files] 
with open('vgg16.yml') as f:
    data = yaml.safe_load(f)
    
d = get_data(data, s, files)
fs.append(d[0])
accs.append(d[1])
scales.append(s)

    
names.append('resnet18')
s = [1]
files = ['resnet18.yml']
files = [(base_path_eval + 'base/' + p) for p in files] 
with open('resnet18.yml') as f:
    data = yaml.safe_load(f)
    
d = get_data(data, s, files)
fs.append(d[0])
accs.append(d[1])
scales.append(s)

    
names.append('prune')
s = list(range(10, 100, 10))
files = ['prune_per_iter=64,percentage={0}.yml'.format(p)
         for p in s]
files = [(base_path_eval + 'prune/' + p) for p in files] 
with open('prune.yml') as f:
    data = yaml.safe_load(f)
    
d = get_data(data, s, files)
fs.append(d[0])
accs.append(d[1])
scales.append(s)


names.append('mimic_teacher')
s = list(range(1, 7))
files = ['learn_from=teacher,sde={0}.yml'.format(p)
         for p in s]
files = [(base_path_eval + 'mimic/' + p) for p in files] 
with open('mimic_teacher.yml') as f:
    data = yaml.safe_load(f)
    
d = get_data(data, s, files)
fs.append(d[0])
accs.append(d[1])
scales.append(s)


names.append('mimic_ground_truth')
s = list(range(1, 7))
files = ['learn_from=ground_truth,sde={0}.yml'.format(p)
         for p in s]
files = [(base_path_eval + 'mimic/' + p) for p in files] 
with open('mimic_ground_truth.yml') as f:
    data = yaml.safe_load(f)
    
d = get_data(data, s, files)
fs.append(d[0])
accs.append(d[1])
scales.append(s)

In [ ]:
def plot_metric_over_speed(names, scales, forward_speeds_all, accs_all):
    d = []
    
    for n, scale, forward_speeds, accs in zip(names, scales, forward_speeds_all, accs_all):
        for m, tmp_fs, tmp_accs in zip(metrics_names, forward_speeds, accs):
            for s, fs, a in zip(scale, tmp_fs, tmp_accs):
                trace = go.Scatter(
                    x = fs,
                    y = a,
                    name = str(s),
                    # legendgroup = o.percentage,
                    mode = 'markers',
                    text = sequences_val
                )
                # d.append(trace)

            x = []
            y = []
            for s, fs, a in zip(scale, tmp_fs, tmp_accs):
                x.append(np.asarray(fs).mean())
                y.append(np.asarray(a).mean())

            trace = go.Scatter(
                x = x,
                y = y,
                text = scale,
                name = n
            )
            d.append(trace)
        
    layout = go.Layout(title='Metric vs Forward Speed',
                       font=dict(family='Roboto'),
                       xaxis=dict(title='Forward Speed in seconds',  ticks='outside'),
                       yaxis=dict(title='Metric value'),
                       showlegend=True,
                      )
    fig = go.Figure(data=d, layout=layout)
    py.iplot(fig)
    
    
plot_metric_over_speed(names, scales, fs, accs)